In [1]:
import tree.ctutils as ctu
from tree import treeutils
import numpy as np
import pickle

# Calculate merger event parameters
def find_merger(atree, idx=None, aexp_min=0.0):
    """
        find indices of merger event from a tree.
        (Full tree or main progenitor trunk)
    """
    if idx == None:
        idx = atree['id'][0]
        
    nprg = 1
    merger_list=[]

    i = 0
    while nprg > 0:
        idx = ctu.get_progenitors(atree, idx, main=True)[0]
        ind = np.where(atree['id'] == idx)[0]
        if atree['aexp'][ind] < aexp_min:
            break
        nprg = ctu.get_npr(atree, idx)

        if nprg > 1:
            merger_list.append(i)
        i +=1
    return merger_list


def merger_mass_ratio(atree, idx=None):
    """
    return mass ratio of the given merger event
    """
    if idx == None:
        idx = atree['id'][0]
        
    prgs = ctu.get_progenitors(atree, idx)
    
    # only for mergers
    if len(prgs) > 1:
        i_prgs = [np.where(atree['id'] == i)[0] for i in prgs]
        mass = []
        for iprg in i_prgs:
            mass.append(atree['m'])
    else:
        print("This is not a merger")
        return 0
    

def merger_properties_main_prg(atree, idx):
    """
        Calculate merger mass ratio for "one" merger event.

    if idx == None:
        if nout == None:
            print("Both idx and nout are missing")
            return
    else:
        if nout == None:
            nout = np.where(atree['id'] == idx)[0]

    idx = atree['id'][ind]
    """    

    #prgs = get_progenitors(atree, idx)
    #if len(prgs) > 1:
    #    i_prgs = [np.where(atree['id'] == i)[0] for i in prgs]
    
    i_prgs = np.where(atree['desc_id'] == idx)[0]
        
    print(i_prgs)
    id_prgs = atree['id'][i_prgs]
    mass_prgs = atree['m'][i_prgs]
    
    #mass_prgs_norm = mass_prgs / sum(mass_prgs)

    return mass_prgs


def load_tree(wdir, is_gal=False, no_dump=False):
    import pickle
    from tree import treemodule
    import tree.ctutils as ctu

    alltrees = treemodule.CTree()
    

    if is_gal:
        # Galaxy tree
        tree_path = 'GalaxyMaker/Trees/'
    else:
        # halo tree
        tree_path = 'halo/Trees/'

    try:
        alltrees = pickle.load(open(wdir + tree_path + "extended_tree.pickle", "rb" ))
        print("Loaded an extended tree")
    except:
        alltrees = treemodule.CTree()
        alltrees.load(filename= wdir + tree_path + 'tree_0_0_0.dat')
        if not no_dump:
            # Fix nout -----------------------------------------------------
            nout_max = alltrees.data['nout'].max()
            alltrees.data['nout'] += 187 - nout_max
            print("------ NOUT fixed")
            alltrees.data = ctu.augment_tree(alltrees.data, wdir, is_gal=is_gal)
            print("------ tree data extended")
        
    return alltrees


In [2]:
import utils.match as mtc
import matplotlib.pyplot as plt
import pandas as pd

        
is_gal = True
# Load tree
wdir = '/home/hoseung/Work/data/05427/'

alltrees = load_tree(wdir, is_gal=True)

Loaded an extended tree


In [3]:
#fig, axs = plt.subplots(11) # 11 subplots in a row]

# Try to do it without catalog.
#catalog = pickle.load(open(wdir + '/catalog_GM/' + 'catalog187.pickle', 'rb'))
#allgals = catalog['id']
nout_fi = 187
ft = alltrees.data[alltrees.data['nout'] == nout_fi]
allgals = ft['id'][ft['m'] > 1e10]

In [ ]:
#fig.add_subplot(222) # == fig.add_subplot(2,2,2)
fig, axs = plt.subplots(2,2)
axr = axs.ravel()
gals = allgals[2:6]
#gals = [42]


verbose = False
cal_lambda = False # requires catalog.pickle



for iax, gal in enumerate(gals):
    if verbose: print("analyzing merger events of galaxy ", gal)

    # Convert halo id to tree id
    #idx = ctu.id_to_idx(alltrees.data, gal, 187)
    idx = gal
    
    # full tree of a galaxy
    atree = ctu.extract_a_tree(alltrees.data, idx)

    # main progenitor tree
    main = ctu.extract_main_tree(alltrees.data, idx)
    if len(main) < 50:
        continue

    x_nout = main['nout'].flatten()

    # all halo ids of main prgs
    main_ids = main['Orig_halo_id']


    # lambda evolution history
    L_r_evol = np.zeros(len(x_nout))
    mass_ratios_single = np.zeros(len(x_nout))

    for i, nout in enumerate(x_nout):
        if nout < 37:
            break
        snout = str(nout)
        if cal_lambda:
            catalog = pickle.load(open(wdir + '/catalog_GM/' + 'catalog' + snout + '.pickle', 'rb'))
            ii = catalog['id'] == main_ids[i]
            if sum(ii) > 0:
                this_gal = catalog[catalog['id'] == main_ids[i]]
                #gal_history.append(this_gal)            
                #L_r_evol[i] = gal_history[i][0]['lambda_r']
                L_r_evol[i] = this_gal['lambda_r'][0]
            else:
                L_r_evol[i]=0

        # merger ratio
        i_prgs = np.where(atree['desc_id'] == main['id'][i])[0]

        # multiple prgs = merger
        if len(i_prgs) > 1:
            if verbose: print(" {} mergers at nout = {} or {}".format(len(i_prgs), nout, main['nout'][i]))
            id_prgs = atree['id'][i_prgs]
            mass_prgs = atree['m'][i_prgs]
            m_r = mass_prgs / max(mass_prgs)
            if verbose:
                print(" Mass ratios : ", m_r)
            mass_ratios_single[i] = max([mass_prgs[1:] / max(mass_prgs)][0])
        else:
            mass_ratios_single[i] = 0

    
    if sum(L_r_evol > 0) < 1:
        # If the galaxy is not identified in any catalog, skip.
        continue

    #if len(gal_history) < 1:
    
    ax = axr[iax]
    ax.set_ylim([0,1.0])
    ax.plot(x_nout, L_r_evol)#[::-1])
    ax.set_title("ID: {0},  $ M_* $: {1:0.1e}".format(str(gal), gal_history[0]['mstar'][0]))
    
    i_merger = mass_ratios_single > 0  
    if sum(i_merger) > 0:
        # Because of the way df.groupby works, sort in increasing order. 
        merger_ratio_bins = [1/100, 1/10, 1/3] # 3:1, 10:1, 100:1 - Major, minor, tiny
        df = pd.DataFrame(data=dict(x=x_nout[i_merger],
                                    y=L_r_evol[i_merger] + 0.3,
                                    q=mass_ratios_single[i_merger]))

        grouped = df.groupby(np.digitize(df.q, merger_ratio_bins))
        sizes = [100 * np.sqrt(i) for i in merger_ratio_bins]
        labels = ["Tiny", "Minor", "Major"] 

        for i, (name, group) in enumerate(grouped):
            ax.scatter(group.x, group.y, s=sizes[i], alpha=0.2 * (1+i), marker='v', label=labels[i], color='red')
        ax.legend()
        
        # print mass ratio only for major mergers
        try:
            a = grouped.get_group(len(merger_ratio_bins) -1) # Last bin = Major Merger
            for x, y, mr in zip(a.x, a.y, a.q):
                ax.annotate("{:.1f}".format(mr),
                            xy=(x, y), xycoords='data',
                            xytext=(0, 5), textcoords='offset points')
        except:
            # There is no Major Merger group.
            pass

plt.show()

In [52]:
main_ids

array([ 48,  56,  42,  51,  23,  36,  19,  44,  21,  49,  66, 111,  44,
        36,  63,  54,  77,  58,  54,  66,  26,  18,  97,  56,  75,  65,
        22,  37,  75,  73,  62,  56,  41,  11, 127,  45,  53, 449, 447,
       476, 471, 472,  52,  46,  33,  38,  89,  76,  28,  28,  53,  85,
        70,  55,  88,  82,  24,  69,  32,  21,  27,  49,  89,  50,  63,
        88,  25,  37, 109, 148,  25,  28,  55,  35,  46,  72, 138,  76,
        41,  46,  34,  44,  94,  26,  79,  85,  82,  70,  97,  49,  99,
       117,  52, 129, 121, 611,  86, 182,  93,  92, 170, 108, 110,  75,
       213,  30, 128,  83,  85,  85, 166, 169,  67,  50,  38,  83,  70,
        60,  55, 106,  86, 136, 160,  98, 195, 200, 243, 167, 160,  96,
       207,  61, 172, 169, 171, 201, 216, 353, 288, 106, 319, 181,  90,
       174, 386, 165, 345, 142, 145, 305, 227, 269,  76, 281, 201, 584,
       244, 367, 147, 256, 371, 303, 373, 325])

In [43]:
catalog['id']

rec.array([ 23,  20,  48,  44,  91,  89, 238, 163, 390, 409, 451,  18,  40,
        86, 133, 379, 438,   4,  32,  56, 100, 364, 417,   9,  34,  63,
       111, 371, 427,  22,  45,  90, 171, 393, 461,  13,  37,  76, 126,
       378, 435,  19,  43,  87, 145, 386, 446,  11,  35,  66, 115, 376,
       431,   2,  31,  53,  97, 363, 415,   7,  33,  61, 106, 369, 426,
         1,  26,  49,  92, 352, 411])

##  Last merger of galaxies

1) divide fast/slow rotators
2) find last merger (mass ration > 10:1)
3) plot rotation paramter Vs mass ratio of last merger..?

In [4]:
#Last merger
import matplotlib.pyplot as plt
nout_ini = 100 # recent merger = nout = 140 or later.
nout_fi = 187


# Load tree
is_gal = True


fig, ax = plt.subplots(1)

# all catalogs


verbose=False

clusters = ['39990', '36415', '10002', '05427', '36413', '01605']
for cluster in clusters:
    wdir = '/home/hoseung/Work/data/' + cluster + '/'
    alltrees = load_tree(wdir, is_gal=True)
    catalog = pickle.load(open(wdir + '/catalog_GM/' + 'catalog187.pickle', 'rb'))
    print("Cluster ",cluster)
    for cat in catalog:
        gal = cat['id']
        if verbose: print("analyzing merger events of galaxy ", gal)

        # Convert halo id to tree id
        #idx = id2idx(alltrees.data, gal, 187)
        idx = cat['idx']

        # full tree of a galaxy
        atree = ctu.extract_a_tree(alltrees.data, idx)

        # main progenitor tree
        main = ctu.extract_main_tree(alltrees.data, idx)

        x_nout = main['nout'].flatten()
        x_nout = x_nout[x_nout > nout_ini]

        mass_ratios_single = np.zeros(len(x_nout))
        for i, nout in enumerate(x_nout):
            # merger ratio
            i_prgs = np.where(atree['desc_id'] == main['id'][i])[0]

            # multiple prgs = merger
            if len(i_prgs) > 1:
                if verbose: print(" {} mergers at nout = {}".format(len(i_prgs), nout))
                id_prgs = atree['id'][i_prgs]
                mass_prgs = atree['m'][i_prgs]
                m_r = mass_prgs / max(mass_prgs)
                if verbose:
                    print(" Mass ratios : ", m_r)
                mass_ratios_single[i] = max([mass_prgs[1:] / max(mass_prgs)][0])
            else:
                mass_ratios_single[i] = 0

        ind_ok = np.where(mass_ratios_single > 0.1)[0]
        #print("all ind_ok", ind_ok)
        if len(ind_ok) > 0:
            ind_ok = max(ind_ok)
            print("  galaxy {}, Last nout  {}, Merger ratio 1:{:.1f}".format(gal,
                                                                         x_nout[ind_ok],
                                                                           1./mass_ratios_single[ind_ok]))
            mr = 1./mass_ratios_single[ind_ok]

            #ax.scatter(x_nout[ind_ok], cat['lambda_r'])
            ax.scatter(x_nout[ind_ok], cat['lambda_r'], s=100*mr)
            ax.set_title("last merger Vs final lambda")
            
ax.set_ylabel(r"$\lambda _R$")
ax.set_xlabel("Last merger")

Loaded an extended tree
Cluster  39990
  galaxy 249, Last nout  186, Merger ratio 1:1.4
  galaxy 364, Last nout  152, Merger ratio 1:4.7
  galaxy 648, Last nout  164, Merger ratio 1:3.7
  galaxy 52, Last nout  102, Merger ratio 1:6.5
  galaxy 160, Last nout  161, Merger ratio 1:4.2
  galaxy 234, Last nout  134, Merger ratio 1:7.9
  galaxy 1987, Last nout  143, Merger ratio 1:1.0
  galaxy 2037, Last nout  139, Merger ratio 1:1.0
  galaxy 104, Last nout  143, Merger ratio 1:1.0
  galaxy 407, Last nout  119, Merger ratio 1:1.0
  galaxy 547, Last nout  138, Merger ratio 1:2.7
  galaxy 165, Last nout  131, Merger ratio 1:2.4
  galaxy 1991, Last nout  144, Merger ratio 1:7.3
  galaxy 127, Last nout  129, Merger ratio 1:9.6
  galaxy 310, Last nout  116, Merger ratio 1:4.4
  galaxy 481, Last nout  113, Merger ratio 1:1.4
  galaxy 677, Last nout  118, Merger ratio 1:2.9
  galaxy 13, Last nout  149, Merger ratio 1:1.0
  galaxy 671, Last nout  105, Merger ratio 1:1.0
  galaxy 1509, Last nout  116

In [5]:
plt.show()

## Delta lambda